In [1]:
from selenium.webdriver import Chrome
from lxml import etree
from time import sleep
from selenium.webdriver.common.keys import Keys
import pandas as pd

# scrape detail/ amenities/ transaction history

In [2]:
def history_info(address):
    web = Chrome()
    web.get("https://www.cityrealty.com/nyc")

    web.find_element_by_xpath('//*[@id="welcome-autocomplete"]').send_keys(address)
    sleep(3)
    el = web.find_element_by_xpath('//*[@id="nbh"]/div[1]/div[1]/div/div[2]/div[2]/cr-welcome-search/div/form/div/div/div/div[3]/div[2]/div/button/span[1]')
    el.click()
    sleep(3)
    
    #get details
    Key_Details=[address]
    li_list1 = web.find_elements_by_xpath('//*[@id="skip-to-content"]/div[2]/div[3]/div[1]/div[1]/div[2]/ul/li[1]/div')
    li_list2 = web.find_elements_by_xpath('//*[@id="skip-to-content"]/div[2]/div[3]/div[1]/div[1]/div[2]/ul/li[2]/div')
    for li in li_list1:
        Key_Details.append(li.find_element_by_xpath("./span[2]").text)
    for li in li_list2:
        Key_Details.append(li.find_element_by_xpath("./span[2]").text)     
     
    #get amenities
    li_list3 = web.find_elements_by_xpath('//*[@class="amenities section"]/ul/li')
    Amenities = []                       
    for li in li_list3:            
        Amenities.append(li.text)

    Key_Details.append(Amenities)
    
    #transaction history
    e2 = web.find_element_by_xpath('//*[@id="skip-to-content"]/div[3]/div/div[2]/button')
    e2.click()
    sleep(2)

    e3 = web.find_element_by_xpath('//*[@id="skip-to-content"]/div[2]/div[3]/div[3]/div[1]/div[2]/div[4]/a')
    e3.click()
    sleep(3)
    
    li_list4 = web.find_elements_by_xpath('//*[@id="bsr_main"]/div[3]/div[2]/div[2]/div')
    hist_data=[]    
    for li in li_list4:
        hist_data.append(li.text)            
    hist = pd.DataFrame(hist_data)
    hist2=hist[0].str.split('\n',expand=True)
    hist2.columns =['Unit','Size','Approx_ft','Closing Price','Price_ft','Asking Price','Closing Date']
    hist2['address']=i

    web.quit()
    return Key_Details, hist2

In [9]:
list_a = []
list_b = []
for i in address:
    try:
        info = history_info(i)
        list_a.append(info[0])
        list_b.append(info[1])
    except Exception:
        pass

In [10]:
df_info = pd.DataFrame(list_a)
df_price = pd.concat(list_b)

In [11]:
df_info

,0,1,2,3,4,5,6,7,8,9
0,455 WEST 19TH STREET,2020,Condominium,Chelsea,10,12,"[Post War, Resident Storage, Pool, Rooftop Ter...",None,None,None
1,80 EAST 10TH STREET,2019,Condominium,East Village,12,10,FT Doorman,"[Attended Lobby, Concierge, PT Doorman, Post W...",None,None
2,300 EAST 71ST STREET,1960,Cooperative,Lenox Hill,284,19,FT Doorman,Allowed,"[Concierge, FT Doorman, Post War, Resident Sto...",None
3,155 EAST 79TH STREET,2016,Condominium,Carnegie Hill,7,14,FT Doorman,"[FT Doorman, Resident Storage, Elevator, Fitne...",None,None
4,15 WEST 61ST STREET,2019,Condominium,Lincoln Center,160,33,FT Doorman,"[Attended Lobby, Concierge, FT Doorman, High-R...",None,None
...,...,...,...,...,...,...,...,...,...,...
91,101 WEST 78TH STREET,1886,Condominium,Broadway Corridor,28,9,FT Doorman,"[Attended Lobby, FT Doorman, Pre War, Children...",None,None
92,801 RIVERSIDE DRIVE,1920,2008,Condominium,Washington Heights,48,6,Allowed,"[Pre War, Elevator, Laundry Room, Live-In Supe...",None
93,70 VESTRY STREET,2018,Condominium,Tribeca,46,14,"[High-Rise, Post War, Resident Storage, Pool, ...",None,None,None
94,143-153 BENNETT AVENUE,1951,Cooperative,Washington Heights,105,6,"[Resident Storage, Full Service Garage, Garden...",None,None,None


In [12]:
df_price

,Unit,Size,Approx_ft,Closing Price,Price_ft,Asking Price,Closing Date,address
0,7,3 beds 2.5 baths,2379,"$7,349,400","$3,089",-,"Mar 3, 2021",455 WEST 19TH STREET
1,4N,2 beds 2.5 baths,1400,"$3,674,700","$2,625",-,"Mar 3, 2021",455 WEST 19TH STREET
2,6N,2 beds 2.5 baths,1219,"$3,092,873","$2,537",-,"Mar 3, 2021",455 WEST 19TH STREET
3,6S,2 beds 2.5 baths,1303,"$3,680,825","$2,825",-,"Feb 24, 2021",455 WEST 19TH STREET
0,7,4 beds 3.5 baths,3304,"$6,400,000 -19.5% from ask price","$1,937","$7,950,000","May 7, 2020",80 EAST 10TH STREET
...,...,...,...,...,...,...,...,...
18,TR59,4 beds 4 baths,2756,"$7,800,000 -27.4% from ask price","$2,830","$10,750,000","May 13, 2020",138 EAST 50TH STREET
19,19D,2 beds 2 baths,1207,"$2,580,000","$2,138","$2,580,000","May 11, 2020",138 EAST 50TH STREET
20,15B,bed bath,0,"$1,551,014",-,-,"May 7, 2020",138 EAST 50TH STREET
21,18D,2 beds 2 baths,1207,"$2,578,000 -9.9% from ask price","$2,136","$2,860,000","Apr 30, 2020",138 EAST 50TH STREET
